Imports

In [1]:
import numpy as np
import pandas as pd
import time
import os
import aux_tools as aux
import Tabesh2013functions as tbsh
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
from collections import defaultdict

### Datos
Se asume que el dataframe tiene una columna para distinguir los clusters de forma global.

In [2]:
mina_df = pd.read_csv("df_mina_clustered_4.csv")
mina_df.rename(columns={'final_cluster_id': 'cluster'}, inplace=True)
mina_df.head()

,x,y,z,au,cpy,cueq,cus,cut,density,material,py,recg_au,recg_cu,tasox,tipomineral,fase,id,banco,cluster,final_cluster_label
0,491425.0,7456205.0,1968.0,0.288929,1.124707,0.712658,0.022003,0.550661,2.615598,8,0.401511,69.001505,87.011938,0.039957,2,4,1,22,1,F4_B22_C1
1,491435.0,7456195.0,1968.0,0.266522,1.057851,0.640223,0.021141,0.491924,2.626056,8,0.197908,68.839492,87.471598,0.042976,2,4,2,22,1,F4_B22_C1
2,491445.0,7456195.0,1968.0,0.243283,2.427782,0.531022,0.019991,0.396348,2.637350,6,0.399009,68.633711,87.659524,0.050438,2,4,3,22,1,F4_B22_C1
3,491455.0,7456195.0,1968.0,0.243283,2.427782,0.531022,0.019991,0.396348,2.637350,6,0.399009,68.633711,87.659524,0.050438,2,4,4,22,1,F4_B22_C1
4,491435.0,7456205.0,1968.0,0.288929,1.124707,0.712658,0.022003,0.550661,2.615598,8,0.401511,69.001505,87.011938,0.039957,2,4,5,22,1,F4_B22_C1


In [3]:
P = 4
R = 0.85 
C_r = 0.25 
C_m = 2 
C_p = 10 
FTL = 2204.62 # Factor tonelada-libra
vol = 1600  # volumen por bloque, puedes ajustar este valor
capacidad_maxima_planta = 5500000
capacidad_maxima_mina = 16000000
# Calcula masa y ley ponderada para cada bloque
mina_df['masa'] = mina_df['density'] * vol
mina_df['fino_total'] = mina_df['masa'] * mina_df['cut']
mina_df['tripleta_fbc'] = list(zip(mina_df['fase'], mina_df['banco'], mina_df['cluster']))
cluster_df = mina_df.groupby('tripleta_fbc').agg({
    'z': 'first',
    'masa': 'sum',
    'fino_total': 'sum'
}).reset_index()

params = {
    row['tripleta_fbc']: {col: row[col] for col in cluster_df.columns if col != 'tripleta_fbc'}
    for _, row in cluster_df.iterrows()
}

Ahora falta hacer las dependencias de la restriccion X~Z

In [8]:
arcs = aux.Global_Vertical_Arc_Calculation(mina_df)
# arcs es un diccionario con llave (f,b,c) y guarda una lista de (f_,b_,c_) indicando los cluster superiores
# Convierte arcs a un defaultdict con ints normales
arcs_clean = defaultdict(list)
for key, values in arcs.items():
    # key limpio
    key_clean = tuple(int(k) for k in key)
    # lista de valores limpios
    values_clean = [tuple(int(v) for v in value) for value in values]
    arcs_clean[key_clean] = values_clean

Ejemplo:

In [9]:
arcs_clean[(1,5,1)]

[(1, 4, 1),
 (1, 4, 2),
 (1, 4, 6),
 (1, 4, 8),
 (1, 4, 9),
 (1, 4, 13),
 (1, 4, 15),
 (2, 4, 11)]

Por tanto, de cierta forma se tiene $$ \mathcal{P}_{f,b,c} = arcs[(f,b,c)]$$

In [16]:
model = Model("PlanificacionMina")
T = list(range(1, 8)) 
r = 0.1
# VARIABLES DE DECISIÓN
x = model.addVars(params.keys(), T, vtype=GRB.BINARY, name="x")
y = model.addVars(params.keys(), T, vtype=GRB.BINARY, name="y")
model.update()
print(len(model.getVars()))


18214


In [17]:
# FUNCIÓN OBJETIVO
model.setObjective(
    quicksum(
        (1/500)*(1 / (1 + r)**t) * (
            ((P - C_r) * R * FTL * params[f, b, c]['fino_total'] *y[f, b, c, t])
            - ((C_p) * params[f, b, c]['masa'] * y[f, b, c, t])
            - (C_m * params[f, b, c]['masa'] * x[f, b, c, t])
        )
        for (f, b, c) in params for t in T
    ), GRB.MAXIMIZE
)

In [18]:
# Restricción de producción anual mínima
for t in T:
    model.addConstr(quicksum(params[f,b,c]['masa']*x[f, b, c, t] for (f, b, c) in params) <= capacidad_maxima_mina)

# Restricción de capacidad de procesamiento anual
for t in T:
    model.addConstr(quicksum(params[f, b, c]['masa'] * y[f, b, c, t] for (f, b, c) in params) <= capacidad_maxima_planta)


# el cluster se extrae una unica vez
for (f, b, c), data in params.items():
        model.addConstr(quicksum(x[f, b, c, t] for t in T) <= 1)


# Relación entre x e y ()
for (f, b, c), data in params.items():
    for t in T:
        model.addConstr(y[f,b,c,t] <= x[f,b,c,t])

# Precedencias
for (f, b, c), data in arcs_clean.items():
    for t in T:
        model.addConstr(len(data)*x[f, b, c, t]  <= quicksum(x[f_, b_, c_, tt] for (f_,b_,c_) in data for tt in range(1, t+1)))


In [19]:

# ==============================
# RESOLVER Y MOSTRAR RESULTADOS
# ==============================
start_time = time.time()

model.optimize()

end_time = time.time()


if model.status == GRB.OPTIMAL:
    print(f"\nVPN óptimo: {model.objVal:.2f} USD")
else:
    print("No se encontró una solución óptima.")

elapsed = end_time - start_time
print(f"\nTiempo de ejecución: {elapsed:.2f} segundos")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2648935 - for non-commercial use only - registered to ig___@usm.cl
Optimize a model with 18738 rows, 18214 columns and 231483 nonzeros
Model fingerprint: 0x249a84e8
Variable types: 0 continuous, 18214 integer (18214 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [7e+00, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 3585 rows and 3585 columns
Presolve time: 0.89s
Presolved: 15153 rows, 14629 columns, 170784 nonzeros
Variable types: 0 continuous, 14629 integer (14629 binary)
Found heuristic solution: objective 4160.4352525
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal lo

In [20]:
model.write('modelo_final.sol')

In [23]:
# Asumiendo que ya tienes:
# - Variables x y y, indexadas como x[f,b,c,t] y y[f,b,c,t]
# - Un diccionario params[(f,b,c)] que tiene 'fino_total' para cada cluster

epsilon = 1e-6

# Crear el resumen
resumen = {}

# Sacamos todos los periodos
periodos = set(t for (_, _, _, t) in x.keys())

for t in sorted(periodos):
    resumen[t] = {
        'extraidos': [],
        'enviados_a_planta': [],
        'fino_procesado': 0.0  # inicializar el fino procesado en el periodo
    }
    for (f, b, c, t_var) in x.keys():
        if t_var == t:
            x_val = x[f, b, c, t].X
            y_val = y[f, b, c, t].X

            if x_val > epsilon:
                resumen[t]['extraidos'].append((f, b, c))
                if y_val > 0.5:
                    resumen[t]['enviados_a_planta'].append((f, b, c))
                    # Sumamos el fino_total asociado a ese cluster
                    fino_cluster = params[(f, b, c)]['fino_total']
                    resumen[t]['fino_procesado'] += fino_cluster

# Ahora imprimimos todo
for t in sorted(resumen.keys()):
    print(f"Periodo {t}:")
    print(f"  Clusters extraídos: {resumen[t]['extraidos']}")
    print(f"  Clusters enviados a planta: {resumen[t]['enviados_a_planta']}")
    print(f"  Fino procesado: {resumen[t]['fino_procesado']:.2f}")
    print("-" * 50)

Periodo 1:
  Clusters extraídos: [(1, 1, 3), (1, 1, 4), (1, 1, 5), (1, 2, 1), (1, 2, 4), (1, 2, 7), (1, 2, 9), (1, 2, 13), (1, 2, 17), (1, 2, 18), (1, 2, 19), (1, 2, 20), (1, 2, 23), (1, 2, 24), (1, 2, 26), (1, 2, 27), (1, 2, 28), (1, 2, 29), (1, 2, 30), (1, 2, 31), (1, 2, 32), (1, 2, 33), (1, 2, 34), (1, 2, 35), (1, 2, 36), (1, 3, 6), (1, 3, 13), (1, 3, 18), (1, 3, 23), (1, 3, 24), (1, 3, 25), (1, 3, 27), (1, 3, 28), (1, 3, 30), (1, 3, 31), (1, 3, 32), (1, 3, 33), (1, 3, 34), (1, 3, 37), (1, 3, 38), (1, 3, 39), (1, 4, 21), (1, 4, 27), (1, 4, 28), (1, 4, 29), (1, 4, 32), (1, 4, 33), (1, 5, 20), (1, 5, 29), (2, 1, 2), (2, 1, 3), (2, 1, 16), (4, 2, 4), (4, 2, 5), (4, 2, 6), (4, 2, 12), (4, 3, 12), (4, 3, 17), (4, 3, 20)]
  Clusters enviados a planta: [(1, 2, 27), (1, 2, 28), (1, 2, 33), (1, 2, 34), (1, 2, 36), (1, 3, 27), (1, 3, 28), (1, 3, 30), (1, 3, 39), (1, 4, 28), (1, 4, 33), (1, 5, 29), (4, 2, 5), (4, 2, 6), (4, 2, 12), (4, 3, 12), (4, 3, 17), (4, 3, 20)]
  Fino procesado: 979816.5